In [1]:
# Load WSC dataset

import xml.etree.ElementTree as etree
# import spacy
import json
# from gensim.models import Word2Vec
import numpy as np
# import pandas as pd
# from pandas import Series
import logging
import numpy
# from nltk.corpus import wordnet as wn
import os


tree = etree.parse('WSCollection.xml')
root = tree.getroot()
original_problems = root.getchildren()
problems = list()

for original_problem in original_problems:
    problem = dict()
    for information in original_problem.getchildren():
        if information.tag == 'answers':
            answers = information.getchildren()
            answer_list = list()
            for answer in answers:
                answer_list.append(answer.text.strip())
            problem['answers'] = answer_list
        elif information.tag == 'text':
            texts = information.getchildren()
            text_dict = dict()
            for text1 in texts:
                text_dict[text1.tag] = text1.text.replace('\n', ' ').strip()
            problem['text'] = text_dict
        elif information.tag == 'quote':
            pass
        else:
            problem[information.tag] = information.text.replace(' ', '')
    problems.append(problem)

print(problems[0])

all_sentences = list()
for question in problems:
    sentence = question['text']['txt1'] + ' ' + question['text']['pron'] + ' ' + question['text']['txt2']
    all_sentences.append(sentence)
    # print(sentence)
    

{'text': {'txt1': 'The city councilmen refused the demonstrators a permit because', 'pron': 'they', 'txt2': 'feared violence.'}, 'answers': ['The city councilmen', 'The demonstrators'], 'correctAnswer': 'A', 'source': '(Winograd1972)'}


/home/guest/miniconda3/envs/gpt2/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/home/guest/miniconda3/envs/gpt2/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/home/guest/miniconda3/envs/gpt2/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/home/guest/miniconda3/envs/gpt2/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [2]:
# Generate GPT answer example
import json
import numpy as np

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
# from scipy.special import softmax
import model, sample, encoder


def softmax(x):
    return np.exp(x)/sum(np.exp(x))

/home/guest/miniconda3/envs/gpt2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/guest/miniconda3/envs/gpt2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/guest/miniconda3/envs/gpt2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/guest/miniconda3/envs/gpt2/lib/python3.7/site-

In [3]:

# def interact_model(
#     problems,
#     answer_collection,
#     full_s_c,
#     keep_original,
#     comparison,
#     model_name='774M',
#     seed=None,
#     nsamples=1,
#     batch_size=5,
#     length=20,
#     temperature=0.1,
#     top_k=10,
#     top_p=1,
#     models_dir= '../models',
# ):
    
#     # models_dir = os.path.expanduser(os.path.expandvars(models_dir))
#     if batch_size is None:
#         batch_size = 1
#     # assert nsamples % batch_size == 0

#     enc = encoder.get_encoder(model_name, models_dir)
#     hparams = model.default_hparams()
#     with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
#         hparams.override_from_dict(json.load(f))

#     if length is None:
#         length = hparams.n_ctx // 2
#     elif length > hparams.n_ctx:
#         raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

#     with tf.Session(graph=tf.Graph()) as sess:
#         context = tf.placeholder(tf.int32, [batch_size, None])
#         np.random.seed(seed)
#         tf.set_random_seed(seed)
#         output = sample.sample_sequence(
#             hparams=hparams, length=length,
#             context=context,
#             batch_size=batch_size,
#             temperature=temperature, top_k=top_k, top_p=top_p
#         )

#         saver = tf.train.Saver()
#         ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
#         saver.restore(sess, ckpt)

#         for i in range(273):
#             question = problems[i]
            
#             real_ans = ""
            
#             if "A" in  problems[i]["correctAnswer"] :
#                 candT = problems[i]["answers"][0]
#                 candF = problems[i]["answers"][1]
#             else:
#                 candT = problems[i]["answers"][1]
#                 candF = problems[i]["answers"][0]
                
#             #  question['text']['pron']
#             candT = candT.replace("The ", "the ")
#             candT = candT.replace("My ", "my ")
#             candF = candF.replace("The ", "the ")
#             candF = candF.replace("My ", "my ")
            
#             if not keep_original:
#                 raw_text = question['text']['txt1'] + ' ' + candT + ' ' + question['text']['txt2'] + ". Because"
#             # raw_text = "The city councilmen refused the demonstrators a permit because the city councilmen feared violence. Because "
#             elif comparison == True:
#                 sentence = question['text']['txt1'] + ' ' + question['text']['pron'] + ' ' + question['text']['txt2']
#                 # relationship = "The '" + question['text']['pron'] +"' is likely to refer to "+candT +" than "+candF+" because"
#                 relationship = "The '" + question['text']['pron'] +"' is less likely to refer to "+ candF+ " than "+candT+" because"
#                 raw_text = sentence + relationship
#             else:
#                 sentence = question['text']['txt1'] + ' ' + question['text']['pron'] + ' ' + question['text']['txt2']
#                 # relationship = "The '" + question['text']['pron'] +"' is likely to refer to "+candT +" than "+candF+" because"
#                 relationship = "The '" + question['text']['pron'] +"' is likely to refer to "+candT +" because"
#                 raw_text = sentence + relationship
            
#             context_tokens = enc.encode(raw_text)
#             generated = 0
#             for _ in range(nsamples // batch_size):
#                 out = sess.run(output, feed_dict={
#                     context: [context_tokens for _ in range(batch_size)]
#                 })[:, len(context_tokens):]
                
#                 for j in range(batch_size):
#                     generated += 1
#                     text = enc.decode(out[j])
                    
#                     stripped_text = text[:(text.rfind(".") + 1)]
                    
#                     print("=" * 40 + " SAMPLE " + str(i) + " " + "=" * 40)
                    
#                     if len(stripped_text) > 0:
#                         text = stripped_text
                    
#                     # print(text)
#                     print(raw_text + text)
#                     full_s_c.append(raw_text + text)
#                     answer_collection.append(text)
            
#             # print("=" * 80)
#         return answer_collection, full_s_c

            
            
# answer_collection = []
# full_s_c = []
# ac, fsc = interact_model(problems, answer_collection, full_s_c, keep_original=True, comparison=False)

In [4]:
# print(fsc)

In [5]:
# with open('full_sent_collection_with_comparison.json', 'w') as f:
#     json.dump(fsc,f)
    
# with open('append_sent_collection_with_comparison.json', 'w') as f:
#     json.dump(ac,f)

In [6]:
import json

import pickle  
import spacy
import json
import numpy as np
#import pandas as pd
import logging
import numpy
import nltk
from nltk.corpus import wordnet as wn
import os

import csv
from tqdm import tqdm_notebook as tqdm

# import en_core_web_sm
#!python -m spacy download en
# !python -m spacy validate
nlp = spacy.load("en")

In [7]:
all_sents = []
for i in range(273):      
    # print(i)
    all_sents.append(problems[i]['text']['txt1'] + ' ' + problems[i]['text']['txt2'])
#     if problems[i]['text']['txt1'] != ".":
#         ans0 = problems[i]['answers'][0].replace("The","the")
#         ans1 = problems[i]['answers'][1].replace("The","the")
#     else:
#         ans0 = problems[i]['answers'][0]
#         ans1 = problems[i]['answers'][1]

#     skeleton1 = problems[i]['text']['txt1'] + ' ' + problems[i]['answers'][0]
#     skeleton2 = problems[i]['text']['txt1'] + ' ' + problems[i]['answers'][1]
#     raw_text1 = problems[i]['text']['txt1'] + ' ' + problems[i]['answers'][0]  + ' ' + problems[i]['text']['txt2']
#     raw_text2 = problems[i]['text']['txt1'] + ' ' + problems[i]['answers'][1] + ' ' +  problems[i]['text']['txt2']

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords
sw = set(stopwords.words('english'))

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize('Eighty-seven miles to go, yet.  Onward!')

# prune a type of POS
def prune_pos(text, target):
    doc = nlp(text)
    temp = []
    count = 0
    for token in doc:
        if token.text == "_":
            temp.append(token.text)
            
        elif token.dep_  in target:
            count = 1
            continue
        else:
            # print(token.dep_)
            temp.append(token.text)
            
    return " ".join(temp[:-1]) + ".", count

def generate_pos_set(all_sents, target):
    all_target = []
    
    for sent in all_sents:
        doc = nlp(sent)
        tars = [token.text for token in doc if token.pos_ == target]
        try:
            tars.remove("_")
        except:
            a = "nothing"
            
        all_target.extend(tars)
    return all_target


def prune_token(text, token):
    temp = text.replace(" "+ token + " "," ")
    temp = temp.replace(token + " "," ")
    temp = temp.replace(" " + token," ")
    
    return temp
    

In [9]:
all_ADP = generate_pos_set(all_sents, "PROPN")

print(all_ADP)

['Joan', 'Susan', 'Joan', 'Susan', 'Paul', 'George', 'Paul', 'George', 'Frank', 'Bill', 'Frank', 'Bill', 'styrofoam', 'John', 'Billy', 'John', 'Billy', 'Tom', 'Ray', 'Tom', 'Ray', 'Sue', 'Sally', 'Sue', 'Sally', 'Sam', 'Tina', 'Sam', 'Tina', 'Anna', 'Lucy', 'Anna', 'Lucy', 'Frank', 'Tom', 'Frank', 'Tom', 'Jim', 'Kevin', 'Jim', 'Kevin', 'Pete', 'Martin', 'Pete', 'Martin', 'Susan', 'Ann', 'Susan', 'Ann', 'Sid', 'Mark', 'Sid', 'Mark', 'Susan', 'Ann', 'Susan', 'Ann', 'Joe', 'Joe', 'Mark', 'Mark', 'Ann', 'Mary', 'Ann', 'Mary', 'Bob', 'Charlie', 'Bob', 'Charlie', 'Bob', 'Charlie', 'Charlie', 'Bob', 'Charlie', 'Charlie', 'Bob', 'Adam', 'Adam', 'Bob', 'Adam', 'Adam', 'Adam', 'Bob', 'Bob', 'Adam', 'Bob', 'Bob', 'Sam', 'Sam', 'Anne', 'Anne', 'Alice', 'Alice', 'Jim', 'Jim', 'keyhole', 'keyhole', 'John', 'John', 'John', 'John', 'Bob', 'Carl', 'Bob', 'Carl', 'Sam', 'Amy', 'Amy', 'Sam', 'Amy', 'Amy', 'Mark', 'Pete', 'Pete', 'Mark', 'Pete', 'Pete', 'Joe', 'Thursday', 'Joe', 'Thursday', 'Paul', 'Paul'

In [10]:
# pruned_tokens = [" this ", " that "]
# pruned_tokens = [","]
# pruned_tokens = ["."]
# pruned_tokens = ["the", "The"]
# pruned_tokens = ["a", "A"]
# pruned_tokens = ["this", "that", "these", "those", "This", "That", "These", "Those"]
# pruned_tokens = ["has", "have", "had", "Has", "Have", "Had"]
# pruned_tokens = ["Because", "because"]
# pruned_tokens = ["go", "going", "went", "gone", "goes"]
# pruned_tokens = ["but", "But"]
# pruned_tokens = ["though", "although", "Though", "Although","since", "Since"] 
# pruned_tokens = ["but", "But", "and", "And"]
# pruned_tokens = ["very", "Very"]
# pruned_tokens = ["so", "So"]
# pruned_tokens = ["in", "In"]
# pruned_tokens = ["of", "Of"]
# pruned_tokens = ["with", "With"]
# pruned_tokens = ["and", "or", "but", "as", "because", "for", "just as", "neither", "nor", "not only", "so", "whether", "yet"]
# pruned_tokens.extend(["And", "Or", "But", "As", "Because", "For", "Just as", "Neither", "Nor", "Not only", "So", "Whether", "Yet"])

pruned_tokens = all_ADP

# CONJS: And As Because But For Just as Or Neither Nor Not only So Whether Yet
all_ADP.append('Kamchatka') #(for names)

# new_ADP = []
# for adp in all_ADP:
#     new_ADP.append(adp.lower())
# pruned_tokens = new_ADP

In [11]:
# print(all_sentences[0])
# print(all_sentences[1])
# print(all_sentences[2])

# print(problems)

import json
import numpy as np
import tensorflow as tf
# from scipy.special import softmax
import model, sample, encoder



model_name = '774M'
models_dir = '../models'

enc = encoder.get_encoder(model_name, models_dir)

batch_size = 1
seed=None
nsamples=1


hparams = model.default_hparams()
with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
    hparams.override_from_dict(json.load(f))
    
length = hparams.n_ctx // 2

answer_collector = []
related_answer_collector = []


def logits_score(logits,skeleton_tokens, context_tokens):
    score = 1
    start_index = len(skeleton_tokens) - 1 
    end_index = len(context_tokens) - 1
    
    for i in range(end_index - start_index): 
        m = softmax(logits[start_index+i])
        score *= m[context_tokens[start_index+i+1]]
    
    return score
        
num_case = 0

with tf.Session(graph=tf.Graph()) as sess:
    
    context = tf.placeholder(tf.int32, [batch_size, None])
    np.random.seed(seed)
    tf.set_random_seed(seed)
    
    context_tokens = []

        # print(context_tokens)

    output = model.model(hparams=hparams, X=context, past=None, reuse=tf.AUTO_REUSE)

    saver = tf.train.Saver()
    ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
    saver.restore(sess, ckpt)

        # out = sess.run(output, feed_dict={context: context_tokens})
        
    for i in range(273):      
        # print(i)
        
        if problems[i]['text']['txt1'] != ".":
            ans0 = problems[i]['answers'][0].replace("The","the")
            ans1 = problems[i]['answers'][1].replace("The","the")
        else:
            ans0 = problems[i]['answers'][0]
            ans1 = problems[i]['answers'][1]
            
        cand0 = problems[i]['answers'][0]
        cand1 = problems[i]['answers'][1]
        
        skeleton1 = problems[i]['text']['txt1'] + ' ' + "_"
        skeleton2 = problems[i]['text']['txt1'] + ' ' + "_"
        raw_text1 = problems[i]['text']['txt1'] + ' ' + "_"  + ' ' + problems[i]['text']['txt2']
        raw_text2 = problems[i]['text']['txt1'] + ' ' + "_"  + ' ' +  problems[i]['text']['txt2']
        
        # Start the pruning process
        
        # pruning the cands (besides names)
        pruned_tokens=[]

        pruned_tokens.extend(tokenizer.tokenize(cand0))
        pruned_tokens.extend(tokenizer.tokenize(cand1))

        names = []
        for name in all_ADP:
            try:
                pruned_tokens.remove(name)
                names.append(name)
            except:
                a = "nothing"
                
        temp_case = 0
        for token in pruned_tokens:
#             skeleton1 = prune_token(skeleton1, token)
#             skeleton2 = prune_token(skeleton2, token)
#             raw_text1 = prune_token(raw_text1, token)
#             raw_text2 = prune_token(raw_text2, token)

            if raw_text1.find(" "+ token + " ") != -1 or raw_text1.find(token + " ") != -1:
                temp_case = 1


        # pruning with dependency structure...
        # "dobj", "pobj"
        # "nsubj"
        # "root"
        # "det"
        # "neg"
        # "amod", "advmod", "npadvmod"
        
#         skeleton1,temp_case = prune_pos(skeleton1, ["neg"])
#         skeleton2,temp_case = prune_pos(skeleton2, ["neg"])
#         raw_text1,temp_case = prune_pos(raw_text1, ["neg"])
#         raw_text2,temp_case = prune_pos(raw_text2, ["neg"])
                
        
        # special case for names
        skeleton1 = skeleton1.replace("_", cand0)
        skeleton2 = skeleton2.replace("_", cand1)
        raw_text1 = raw_text1.replace("_", cand0)
        raw_text2 = raw_text2.replace("_", cand1)
        

        
        num_case +=  temp_case
        
        context_tokens1 = enc.encode(raw_text1)
        context_tokens2 = enc.encode(raw_text2)
        skeleton_tokens1 = enc.encode(skeleton1)
        skeleton_tokens2 = enc.encode(skeleton2)
        
        out1 = sess.run(output, feed_dict={context: [context_tokens1 for _ in range(batch_size)]})
        out2 = sess.run(output, feed_dict={context: [context_tokens2 for _ in range(batch_size)]})
        
        logits1 = out1['logits'][:, :, :hparams.n_vocab]
        logits2 = out2['logits'][:, :, :hparams.n_vocab]
        # presents = out['present']    
    
            
        score1 = logits_score(logits1[0],skeleton_tokens1,context_tokens1)
        score2 = logits_score(logits2[0],skeleton_tokens2,context_tokens2)
        
        
        correctAnswer = problems[i]["correctAnswer"]

        if score1 >= score2:
            predictedAnswer = "A"
        else:
            predictedAnswer = "B"
        # A. Problem
        answer_collector.append(predictedAnswer in correctAnswer)
        
print(num_case)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../models/774M/model.ckpt
158


In [12]:
print(np.sum(answer_collector))
print(np.sum(answer_collector)/273)

189
0.6923076923076923


In [15]:
# print(answer_collector)

In [13]:
with open("gpt2_answers.json", "w", encoding="utf-8") as f:
    json.dump(answer_collector,f)